In [1]:
!nvidia-smi

Thu May 23 09:34:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   49C    P8              13W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split




### Set up device agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
# Setup manual transforms

manual_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [4]:
# get food101 data
dataset = datasets.Food101(root='./data', transform=manual_transforms, download=True)

100%|██████████| 4996278331/4996278331 [04:28<00:00, 18623972.00it/s]


Extracting ./data/food-101.tar.gz to ./data


In [13]:
### Getting a subset of the dataset with only 10 classes
from torch.utils.data import Subset

selected_classes = ['apple_pie', 'baby_back_ribs', 'baklava',
                    'beef_carpaccio', 'beef_tartare', 'beet_salad',
                    'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito']


selected_indices = [dataset.class_to_idx[class_name] for class_name in selected_classes]


indices_to_keep = [i for i, (_, label) in enumerate(dataset) if label in selected_indices]
filtered_dataset = Subset(dataset, indices_to_keep)


print(f"Number of samples in the filtered dataset: {len(filtered_dataset)}")

Number of samples in the filtered dataset: 7500


In [19]:
## Splitting data into train, validation and test

train = int(0.8 * len(filtered_dataset))
validation = int(0.1 * len(filtered_dataset))
test = int(0.1 * len(filtered_dataset))

train_data, validation_data, test_data = random_split(filtered_dataset, [train, validation, test])

In [20]:
## preparing dataloaders

train_dataloader = DataLoader(train_data, batch_size = 32, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=False)

In [21]:
## Loading the pretained model and modifying the classfier layer's out_features

model = models.efficientnet_b0()
model = model.to(device)


model.classifier[1] = nn.Linear(model.classifier[1].in_features, out_features=10)

In [22]:
## setting up loss_function and optimizer n accuracy function

loss_function = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(params=model.parameters(), lr=0.001)

def accuracy_function(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

In [23]:
## setting up training function and validation function
from tqdm.auto import tqdm
def train_function(model, device, train_dataloader, loss_function, optimizer):
  model.train()
  model = model.to(device)
  training_loss = 0
  for X_train, y_train in tqdm(train_dataloader):
    X_train, y_train = X_train.to(device), y_train.to(device)
    y_train_logits = model(X_train)
    loss = loss_function(y_train_logits, y_train)
    training_loss += loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  # find average loss per batch
  return (training_loss/len(train_dataloader))


def validate_function(model, device, validation_dataloader, loss_function, accuracy_function):
  model.eval()
  model = model.to(device)
  validation_loss = 0
  validation_acc = 0

  with torch.inference_mode():
    for X_val, y_val in tqdm(validation_dataloader):
      X_val , y_val = X_val.to(device), y_val.to(device)
      val_logits = model(X_val)
      val_pred = torch.argmax(val_logits, dim=1)
      validation_acc += accuracy_function(y_val, val_pred)
      validation_loss += loss_function(val_logits, y_val)
  return validation_loss/len(validation_dataloader), validation_acc/len(validation_dataloader)



In [24]:
## Actually training and validating the model

epochs = 5
for epoch in range(epochs):
  train_loss = train_function(model, device, train_dataloader, loss_function, optimizer)
  validation_loss, validation_acc = validate_function(model, device, validation_dataloader, loss_function, accuracy_function)
  print(f'Epoch {epoch}: Training Loss: {train_loss:.4f}, Validation Loss: {validation_loss:.4f}, Validation Accuracy: {validation_acc:.2f}%')

  0%|          | 0/188 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

Epoch 0: Training Loss: 2.2868, Validation Loss: 2.2654, Validation Accuracy: 13.52%


  0%|          | 0/188 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

...

  0%|          | 0/24 [00:00<?, ?it/s]

Epoch 3: Training Loss: 2.2184, Validation Loss: 2.1851, Validation Accuracy: 15.83%


  0%|          | 0/188 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

Epoch 4: Training Loss: 2.2018, Validation Loss: 2.1751, Validation Accuracy: 17.47%
